<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/363_Prioritization_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Prioritization & Decision Support

### Turning Patterns Into Clear Action

This code block is where the Employee Feedback Intelligence Agent answers the most important question leaders ask:

> **“What should we focus on first?”**

Rather than relying on intuition, volume alone, or opaque AI scoring, this prioritization layer uses **explicit, business-aligned rules** grounded in the **Pareto (80/20) principle** to surface the issues and ideas that matter most.

The result is not just ranked feedback — it’s **explainable prioritization**.

---

## Why Prioritization Is Treated as a Business Decision

Many systems jump straight from sentiment or themes to recommendations.
This agent deliberately inserts a prioritization layer in between.

Why?

Because leaders don’t want:

* Raw complaints
* Emotional summaries
* Long lists with no ordering

They want:

* Clear tradeoffs
* Justified focus areas
* Confidence that attention is being spent wisely

This layer provides exactly that.

---

## How Priority Is Calculated (Conceptually)

### `calculate_priority_score`

Each feedback entry receives a **priority score from 0–100**, based on three factors leaders intuitively understand:

#### 1. Frequency (Pattern Strength)

If a piece of feedback is part of a recurring theme:

* It receives more weight
* Repeated issues matter more than one-offs

This ensures the system focuses on **systemic problems**, not isolated anecdotes.

---

#### 2. Sentiment Intensity (Pressure Signal)

Sentiment doesn’t decide priority — but it *modulates* it.

* Strong negative sentiment increases urgency
* Strong positive sentiment highlights high-impact ideas
* Neutral sentiment keeps focus on facts

This captures **how painful or energizing** an issue feels without letting emotion dominate logic.

---

#### 3. Category Weight (Urgency vs Opportunity)

The system recognizes that:

* **Issues** usually require faster attention
* **Ideas** represent opportunity, not emergency

Both are important — but not equally urgent.

This weighting mirrors how managers naturally triage work.

---

## Separate Pipelines for Issues and Ideas (On Purpose)

### `prioritize_issues`

Issues are prioritized based on:

* How often they occur
* How intense the frustration is
* Whether they represent operational risk

The output is a ranked list of **problems most likely to cause friction if ignored**.

---

### `prioritize_ideas`

Ideas are prioritized based on:

* Recurrence across employees
* Positive sentiment strength
* Potential impact

This allows leadership to spot:

* Low-effort, high-impact improvements
* Repeated suggestions pointing to easy wins
* Areas of strong frontline engagement

Issues and ideas are evaluated using the *same framework* — but interpreted differently.

That’s intentional.

---

## Human-Readable Rationale (No Black Boxes)

### `_generate_prioritization_rationale`

Every prioritized item includes a short explanation of **why it ranked highly**.

Examples:

* “High frequency theme (8 occurrences) | strong negative sentiment | operational issue requiring attention”
* “High frequency theme (5 occurrences) | strong positive sentiment | improvement idea with potential impact”

This means:

* Leaders don’t have to trust the score blindly
* Decisions are defensible
* Conversations stay grounded in evidence

If someone asks *“Why are we focusing on this?”* — the answer is built in.

---

## Pareto Thinking Without Rigid Math

The agent applies the **spirit** of the 80/20 principle without forcing artificial thresholds.

Instead of saying:

* “Exactly 20% of issues matter”

It asks:

* “Which small set of issues is clearly driving most of the friction?”

This flexibility keeps prioritization realistic, human, and aligned with real-world decision-making.

---

## Executive Summary of Priorities

### `calculate_prioritization_summary`

This function produces a clean leadership snapshot:

* Top 3 issues
* Top 3 ideas
* Average priority scores
* Volume of prioritized items

This is what feeds:

* Executive summaries
* Dashboards
* Weekly or monthly leadership reviews

It’s designed to answer:

> “What’s most important right now — and is that changing?”

---

## Why Leaders Can Trust This

From a governance perspective, this prioritization model:

* Makes tradeoffs explicit
* Avoids opaque AI scoring
* Keeps logic adjustable
* Aligns with how leaders already think

It’s not replacing judgment — it’s **supporting it with structure**.

---

## Architectural Takeaway

This prioritization layer is where insight becomes action.

By combining frequency, sentiment, and category intent into a transparent scoring model — and by explaining every result — the agent delivers **clarity without automation overreach**.

That’s what makes this system usable, scalable, and credible at the executive level.




# Prioritization Utilities for Employee Feedback Intelligence Agent

In [ ]:
"""Prioritization Utilities for Employee Feedback Intelligence Agent

Uses Pareto (80/20) principle conceptually to identify top issues and ideas.
"""

from typing import List, Dict, Any, Optional
from collections import defaultdict


def calculate_priority_score(
    entry: Dict[str, Any],
    theme: Optional[Dict[str, Any]] = None,
    sentiment_analysis: Optional[Dict[str, Any]] = None
) -> float:
    """
    Calculate priority score for a feedback entry.

    Priority is based on:
    - Frequency (if part of a theme)
    - Sentiment intensity
    - Category (Issues typically higher priority than Ideas)

    Args:
        entry: Feedback entry
        theme: Optional theme this entry belongs to
        sentiment_analysis: Optional sentiment analysis for this entry

    Returns:
        Priority score (0-100)
    """
    score = 0.0

    # Base score from frequency (if part of theme)
    if theme:
        frequency = theme.get("frequency", 1)
        score += min(40.0, frequency * 5.0)  # Up to 40 points for frequency

    # Sentiment intensity (negative issues get higher priority)
    if sentiment_analysis:
        intensity = sentiment_analysis.get("intensity", 0.5)
        sentiment = sentiment_analysis.get("sentiment", "neutral")

        if sentiment == "negative":
            score += intensity * 30.0  # Up to 30 points for negative sentiment
        elif sentiment == "positive":
            score += intensity * 15.0  # Up to 15 points for positive sentiment

    # Category weight (Issues typically more urgent)
    category = entry.get("category", "Unknown")
    if category == "Issue":
        score += 20.0
    elif category == "Idea":
        score += 10.0

    # Cap at 100
    return min(100.0, round(score, 2))


def prioritize_issues(
    feedback: List[Dict[str, Any]],
    themes: List[Dict[str, Any]],
    sentiment_analysis: List[Dict[str, Any]],
    top_n: int = 10
) -> List[Dict[str, Any]]:
    """
    Prioritize issues using frequency, sentiment, and impact.

    Args:
        feedback: List of feedback entries
        themes: List of detected themes
        sentiment_analysis: List of sentiment analysis results
        top_n: Number of top issues to return

    Returns:
        List of prioritized issues sorted by priority score
    """
    # Create lookups
    sentiment_lookup = {s["submission_id"]: s for s in sentiment_analysis}
    theme_lookup = {}
    for theme in themes:
        for feedback_id in theme.get("feedback_ids", []):
            if feedback_id not in theme_lookup:
                theme_lookup[feedback_id] = theme

    # Calculate priority for each issue
    issues = [e for e in feedback if e.get("category") == "Issue"]
    prioritized = []

    for entry in issues:
        submission_id = entry.get("submission_id")
        theme = theme_lookup.get(submission_id)
        sentiment = sentiment_lookup.get(submission_id, {})

        priority_score = calculate_priority_score(entry, theme, sentiment)

        prioritized.append({
            "submission_id": submission_id,
            "theme_id": theme.get("theme_id") if theme else None,
            "priority_score": priority_score,
            "frequency": theme.get("frequency", 1) if theme else 1,
            "sentiment_intensity": sentiment.get("intensity", 0.5),
            "department": entry.get("job_area", "Unknown"),
            "category": entry.get("category", "Issue"),
            "feedback_text": entry.get("free_text_feedback", ""),
            "rationale": _generate_prioritization_rationale(entry, theme, sentiment, priority_score)
        })

    # Sort by priority score (descending)
    prioritized.sort(key=lambda x: x["priority_score"], reverse=True)

    return prioritized[:top_n]


def prioritize_ideas(
    feedback: List[Dict[str, Any]],
    themes: List[Dict[str, Any]],
    sentiment_analysis: List[Dict[str, Any]],
    top_n: int = 10
) -> List[Dict[str, Any]]:
    """
    Prioritize ideas using frequency, sentiment, and potential impact.

    Args:
        feedback: List of feedback entries
        themes: List of detected themes
        sentiment_analysis: List of sentiment analysis results
        top_n: Number of top ideas to return

    Returns:
        List of prioritized ideas sorted by priority score
    """
    # Create lookups
    sentiment_lookup = {s["submission_id"]: s for s in sentiment_analysis}
    theme_lookup = {}
    for theme in themes:
        for feedback_id in theme.get("feedback_ids", []):
            if feedback_id not in theme_lookup:
                theme_lookup[feedback_id] = theme

    # Calculate priority for each idea
    ideas = [e for e in feedback if e.get("category") == "Idea"]
    prioritized = []

    for entry in ideas:
        submission_id = entry.get("submission_id")
        theme = theme_lookup.get(submission_id)
        sentiment = sentiment_lookup.get(submission_id, {})

        priority_score = calculate_priority_score(entry, theme, sentiment)

        prioritized.append({
            "submission_id": submission_id,
            "theme_id": theme.get("theme_id") if theme else None,
            "priority_score": priority_score,
            "frequency": theme.get("frequency", 1) if theme else 1,
            "sentiment_intensity": sentiment.get("intensity", 0.5),
            "department": entry.get("job_area", "Unknown"),
            "category": entry.get("category", "Idea"),
            "feedback_text": entry.get("free_text_feedback", ""),
            "rationale": _generate_prioritization_rationale(entry, theme, sentiment, priority_score)
        })

    # Sort by priority score (descending)
    prioritized.sort(key=lambda x: x["priority_score"], reverse=True)

    return prioritized[:top_n]


def _generate_prioritization_rationale(
    entry: Dict[str, Any],
    theme: Optional[Dict[str, Any]],
    sentiment: Dict[str, Any],
    priority_score: float
) -> str:
    """Generate human-readable rationale for prioritization."""
    parts = []

    if theme:
        parts.append(f"High frequency theme ({theme.get('frequency', 1)} occurrences)")

    sentiment_type = sentiment.get("sentiment", "neutral")
    intensity = sentiment.get("intensity", 0.5)
    if sentiment_type == "negative" and intensity > 0.7:
        parts.append("strong negative sentiment")
    elif sentiment_type == "positive" and intensity > 0.7:
        parts.append("strong positive sentiment")

    category = entry.get("category", "Unknown")
    if category == "Issue":
        parts.append("operational issue requiring attention")
    elif category == "Idea":
        parts.append("improvement idea with potential impact")

    if not parts:
        return f"Priority score: {priority_score:.1f}"

    return " | ".join(parts)


def calculate_prioritization_summary(
    prioritized_issues: List[Dict[str, Any]],
    prioritized_ideas: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Calculate summary of prioritization results.

    Args:
        prioritized_issues: List of prioritized issues
        prioritized_ideas: List of prioritized ideas

    Returns:
        Summary dictionary
    """
    return {
        "top_3_issues": prioritized_issues[:3],
        "top_3_ideas": prioritized_ideas[:3],
        "total_prioritized_issues": len(prioritized_issues),
        "total_prioritized_ideas": len(prioritized_ideas),
        "average_issue_priority": sum(i["priority_score"] for i in prioritized_issues) / len(prioritized_issues) if prioritized_issues else 0.0,
        "average_idea_priority": sum(i["priority_score"] for i in prioritized_ideas) / len(prioritized_ideas) if prioritized_ideas else 0.0
    }

